In [3]:
# --- IMPORTS ---
import cv2
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn import preprocessing


ModuleNotFoundError: No module named 'cv2'

In [4]:
import sys
!{sys.executable} -m pip install opencv-python opencv-contrib-python pillow numpy matplotlib seaborn scipy scikit-learn


  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl (45.3 MB)

   ---------------------------------------- 0/2 [opencv-python]
   ---------------------------------------- 0/2 [opencv-python]
   ---------------------------------------- 0/2 [opencv-python]
   -------------------- ------------------- 1/2 [opencv-contrib-python]
   -------------------- ------------------- 1/2 [opencv-contrib-python]
   -------------------- ------------------- 1/2 [opencv-contrib-python]
   -------------------- ------------------- 1/2 [opencv-contrib-python]
   ---------------------------------------- 2/2 [opencv-contrib-python]



In [1]:
# --- IMPORTS ---
import cv2
import numpy as np
from PIL import Image, ImageOps


# --- STEP 1: Load the Image ---
img_path = "distorted_qr.png"  # replace with your file path
image = Image.open(img_path)


# Convert to grayscale for simplicity
gray = ImageOps.grayscale(image)
img_cv = np.array(gray)


# --- STEP 2: Denoising ---
# Remove small specks/noise while keeping edges
denoised = cv2.medianBlur(img_cv, 3)


# --- STEP 3: Contrast Enhancement ---
# Improves visibility of the black/white QR blocks
enhanced = cv2.convertScaleAbs(denoised, alpha=2.5, beta=0)


# --- STEP 4: Adaptive Thresholding ---
# Binarizes the image (turns it pure black & white)
binary = cv2.adaptiveThreshold(
    enhanced, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY, 35, 10
)


# --- STEP 5: Morphological Cleaning ---
# Removes noise & smooths block edges
kernel = np.ones((3, 3), np.uint8)
morph = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)
morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)


# --- STEP 6: Try to Decode the QR Code ---
detector = cv2.QRCodeDetector()


# Try decoding several filtered versions for robustness
attempts = [img_cv, denoised, enhanced, binary, morph]
for idx, img in enumerate(attempts, 1):
    data, points, _ = detector.detectAndDecode(img)
    if data:
        print(f"✅ QR Decoded at step {idx}:", data)
        break
else:
    print("❌ QR not decoded — try adjusting alpha/beta or block size in thresholding.")

✅ QR Decoded at step 2: SIGMOID_{cl4551c_cv_w1n5_S1gM014_N1C3_4N4Lys15_7r1ck5}
